In [1]:
!pip3 install tensorflow_text>=2.0.0rc0

In [ ]:
!pip install transformers sentencepiece

In [12]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from collections import Counter
import time
import torch

In [6]:
df_no_bal = pd.read_json('/content/drive/MyDrive/cw_classifier/parsed.jl', lines=True)
df_no_bal['rating'] = df_no_bal['rating']/10
df_no_bal.drop(['hotel', 'address', 'date'], axis=1, inplace=True)
df_no_bal.rename(columns={'content': 'review'}, inplace=True)
grouped_df = df_no_bal.groupby('rating')
dfs_by_rating = [grouped_df.get_group(x) for x in grouped_df.groups]
min_count = 12000
dfs_sampled = []
for d in dfs_by_rating:
  if len(d) >= min_count:
    dfs_sampled.append(d.sample(min_count, random_state=42))
  else:
    dfs_sampled.append(d.sample(len(d), random_state=42))
# dfs_sampled = [d.sample(min_count, random_state=42) for d in dfs_by_rating]
df_low2 = pd.concat(dfs_sampled)

In [9]:
Counter(df_low2['rating'])

Counter({1.0: 8082, 2.0: 7050, 3.0: 12000, 4.0: 12000, 5.0: 12000})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

In [13]:
sentences = df_low2['review'].to_list()
total_embed = []
start_time = time.time()
for i in range(0, len(sentences), 200):
  if i % 2000 == 0:
    print('i =', i, '\ntime:', time.time() - start_time, '\n')
  if i+200 > len(sentences):
    encoded_input = tokenizer(sentences[i:], padding=True, truncation=True, max_length=64, return_tensors='pt')
  else:
    encoded_input = tokenizer(sentences[i:i+200], padding=True, truncation=True, max_length=64, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  embeddings = model_output.pooler_output
  embeddings = torch.nn.functional.normalize(embeddings)
  total_embed += embeddings.tolist()
df_low2['LaBASE'] = total_embed

i = 0 
time: 0.00040531158447265625 

i = 2000 
time: 473.3898251056671 

i = 4000 
time: 945.9960858821869 

i = 6000 
time: 1424.5306441783905 

i = 8000 
time: 1906.5493898391724 

i = 10000 
time: 2384.4580121040344 

i = 12000 
time: 2853.8977394104004 

i = 14000 
time: 3330.947544336319 

i = 16000 
time: 3801.8210158348083 

i = 18000 
time: 4270.6061668396 

i = 20000 
time: 4736.09121966362 

i = 22000 
time: 5203.938910245895 

i = 24000 
time: 5671.57924079895 

i = 26000 
time: 6136.25023150444 

i = 28000 
time: 6602.400271177292 

i = 30000 
time: 7063.6029324531555 

i = 32000 
time: 7521.620379924774 

i = 34000 
time: 7990.369554758072 

i = 36000 
time: 8450.882364749908 

i = 38000 
time: 8914.683479070663 

i = 40000 
time: 9374.331977128983 

i = 42000 
time: 9832.143565654755 

i = 44000 
time: 10290.317581176758 

i = 46000 
time: 10747.55672621727 

i = 48000 
time: 11203.931102752686 

i = 50000 
time: 11661.986273288727 



In [14]:
df_low2.to_csv('Hotel_such_no_bal.csv')